In [1]:
import cv2
import os
import time
from pymodbus.client import ModbusTcpClient

In [2]:
coil_address = 1542 #ModBus Y0
input_address = 0
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

In [3]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")
    return client

In [4]:
def read_PLC_output(client, coil_address):
    read_result = client.read_coils(address=coil_address)  # Read a single coil (output)
    if read_result.isError():
        print("Read Error:", read_result)
        return None
    return read_result.bits[0]  # Return the first coil state (True for ON, False for OFF)


In [5]:
def capture(client):
    cap = cv2.VideoCapture(2)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    output_dir = "frames"
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    frame_count = 0
    flag = False  # Tracks the previous state of the output

    while True:
        result = client.read_coils(1542)  # Ensure count=1 to avoid errors
        if result.isError():
            print("Modbus Read Error:", result)
            continue  # Skip iteration on error

        current_state = result.bits[0]  # Read the coil status
        print("Output status:", current_state)

        # Detect rising edge: transition from False -> True
        if current_state and not flag:
            #time.sleep(0.3)
            print("TRUE - Capturing Frame")
            
            ret, frame = cap.read()
            if not ret:
                print("Error: Frame capture failed.")
                break
            
            # Save the frame
            frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}") 
            frame_count += 1

            # Display the frame
            cv2.imshow("Live Stream - Capturing Frames", frame)
            #time.sleep(0.5)  # Small delay to avoid duplicate captures

        # Update flag to track previous state
        flag = current_state

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


In [6]:
if __name__ == "__main__":

    client = modbus(PLC_IP, PLC_PORT)
    capture(client)

Connection successful
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output statu